First lets do the avg-based FFT, based on these basis functions:


$\bar{s}^k_i = \left< sin(2 \pi k x) \right>_i$

$\bar{c}^k_i = \left< cos(2 \pi k x) \right>_i$

where $x_i = i / N$ for $i \in [0, \dots, N-1]$, and  $\left< \cdot\right>_i$ means average over $[x_i,x_{i+1}]$

In [1]:
import numpy as np
import numpy.fft as fft

# NOTE - these lines will shut off wrapping so scrollbar instead
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))
np.set_printoptions( linewidth=1000)

In [2]:
# Number of points in domain - don't forget x=1 is periodic wrap of x=0
N=8

In [3]:
# Create the node-based functions
x_node = np.linspace(0,1-1/N,N)  # x node values from 0 ... not including 1
dx = 1/N  # cell spacing
basis_k = np.zeros((N,N))  # basis functions
basis_k[:,0] = np.ones(N)  # k=0 mode
key_k = ['c0']  # labels for modes
k_vals = np.zeros(N)  # for k numbers
# Fill in sin/cos modes for k between 0 and N/2
x_lo = x_node[0:N]  # left cell edges
x_hi = np.append(x_node[1:N],[1])  # right cell edges

In [4]:
# Initialize the avg of each basis
for k in range(1,int(N/2)):
    k_vals[2*k-1] = k
    k_vals[2*k] = k
    key_k.append(f's{k}')
    key_k.append(f'c{k}')
    tpik = 2*np.pi*k
    basis_k[:,2*k-1] = -1/(tpik*dx)*(np.cos(tpik*x_hi) - np.cos(tpik*x_lo))
    basis_k[:,2*k] = 1/(tpik*dx)*(np.sin(tpik*x_hi) - np.sin(tpik*x_lo))
basis_k[:,N-1] = (2/np.pi)*(1 - 2*np.mod(np.array(range(0,N)),2))  # cell avg of +/- highest sin mode
key_k.append(f's{int(N/2)}') 
k_vals[N-1]=int(N/2)
basis_k.T  # each row is a mode for coef 1 in front of that sign wave

array([[ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,  1.        ,  1.        ,  1.        ],
       [ 0.37292323,  0.90031632,  0.90031632,  0.37292323, -0.37292323, -0.90031632, -0.90031632, -0.37292323],
       [ 0.90031632,  0.37292323, -0.37292323, -0.90031632, -0.90031632, -0.37292323,  0.37292323,  0.90031632],
       [ 0.63661977,  0.63661977, -0.63661977, -0.63661977,  0.63661977,  0.63661977, -0.63661977, -0.63661977],
       [ 0.63661977, -0.63661977, -0.63661977,  0.63661977,  0.63661977, -0.63661977, -0.63661977,  0.63661977],
       [ 0.72451862, -0.30010544, -0.30010544,  0.72451862, -0.72451862,  0.30010544,  0.30010544, -0.72451862],
       [ 0.30010544, -0.72451862,  0.72451862, -0.30010544, -0.30010544,  0.72451862, -0.72451862,  0.30010544],
       [ 0.63661977, -0.63661977,  0.63661977, -0.63661977,  0.63661977, -0.63661977,  0.63661977, -0.63661977]])

In [5]:
print(key_k)
print(k_vals)

['c0', 's1', 'c1', 's2', 'c2', 's3', 'c3', 's4']
[0. 1. 1. 2. 2. 3. 3. 4.]


In [6]:
# Create a Laplacian
ones = np.full(N,1)
lapl = np.diagflat(-2*ones,0) + np.diagflat(ones[1:N],1) \
       + np.diagflat(ones[1:N],-1)
lapl[0,N-1] = 1
lapl[N-1,0] = 1
print('Laplacian = \n' + str(lapl))

Laplacian = 
[[-2  1  0  0  0  0  0  1]
 [ 1 -2  1  0  0  0  0  0]
 [ 0  1 -2  1  0  0  0  0]
 [ 0  0  1 -2  1  0  0  0]
 [ 0  0  0  1 -2  1  0  0]
 [ 0  0  0  0  1 -2  1  0]
 [ 0  0  0  0  0  1 -2  1]
 [ 1  0  0  0  0  0  1 -2]]


In [7]:
# See what L * our modes are, and the eigenvalues
Lv_ratio = np.zeros((N,N))
eigvals = np.zeros(N)
for k in range(0,N):
    Lv = lapl@basis_k[:,k]
    Lv_ratio[k,:] = Lv / basis_k[:,k]
    # This will make sure all the entry-wise values are the same
    if np.isclose(Lv_ratio[k,:], Lv_ratio[k,0], 1e-14).all():
        eigvals[k] = Lv_ratio[k,0]

print(eigvals)
test_eigvals = -2 + 2*np.cos(2*np.pi*k_vals/N)
# Check theoretical eigenvalues vs. what we calculated from cell-averages
assert np.isclose(eigvals, test_eigvals, 1e-14).all()

print('\nLaplacian eigenvector/value test passed!')

[ 0.         -0.58578644 -0.58578644 -2.         -2.         -3.41421356 -3.41421356 -4.        ]

Laplacian eigenvector/value test passed!


In [8]:
# Show they're orthogonal, too
orth_k = np.round(basis_k.T@basis_k,14)
print(orth_k)

# NOTE: we need these for coef scaling on the transform!
scale_k = np.diag(orth_k)
scale_k

[[ 8.          0.         -0.         -0.         -0.          0.         -0.          0.        ]
 [ 0.          3.79856481 -0.          0.          0.          0.          0.         -0.        ]
 [-0.         -0.          3.79856481 -0.         -0.          0.         -0.         -0.        ]
 [-0.          0.         -0.          3.24227788 -0.          0.         -0.          0.        ]
 [-0.          0.         -0.         -0.          3.24227788  0.         -0.         -0.        ]
 [ 0.          0.          0.          0.          0.          2.45996202 -0.          0.        ]
 [-0.          0.         -0.         -0.         -0.         -0.          2.45996202 -0.        ]
 [ 0.         -0.         -0.          0.         -0.          0.         -0.          3.24227788]]


array([8.        , 3.79856481, 3.79856481, 3.24227788, 3.24227788, 2.45996202, 2.45996202, 3.24227788])

In [9]:
# For example, let's test for some combination of modes
y = 1*basis_k[:,0] - 2*basis_k[:,1] + .5*basis_k[:,N-1]
# Another way to create this is with matrix mult
test_coef = np.zeros(N)
test_coef[0] = 1
test_coef[1] = -2
test_coef[N-1] = .5
test_y = test_coef@basis_k.T
assert np.isclose(y, test_y, 1e-14).all()

# This is the official representation of our fv_fft as a matrix
fv_fft = np.diagflat(1/scale_k)@basis_k.T
print('FV_FFT = ')
print(fv_fft)
# coef = (basis_k.T@y) / scale_k
coef = fv_fft@y
print(np.round(coef, 14))
assert np.isclose(coef, test_coef, 1e-14).all()

print('\nTests passed')

FV_FFT = 
[[ 0.125       0.125       0.125       0.125       0.125       0.125       0.125       0.125     ]
 [ 0.09817477  0.23701486  0.23701486  0.09817477 -0.09817477 -0.23701486 -0.23701486 -0.09817477]
 [ 0.23701486  0.09817477 -0.09817477 -0.23701486 -0.23701486 -0.09817477  0.09817477  0.23701486]
 [ 0.19634954  0.19634954 -0.19634954 -0.19634954  0.19634954  0.19634954 -0.19634954 -0.19634954]
 [ 0.19634954 -0.19634954 -0.19634954  0.19634954  0.19634954 -0.19634954 -0.19634954  0.19634954]
 [ 0.29452431 -0.12199596 -0.12199596  0.29452431 -0.29452431  0.12199596  0.12199596 -0.29452431]
 [ 0.12199596 -0.29452431  0.29452431 -0.12199596 -0.12199596  0.29452431 -0.29452431  0.12199596]
 [ 0.19634954 -0.19634954  0.19634954 -0.19634954  0.19634954 -0.19634954  0.19634954 -0.19634954]]
[ 1.  -2.   0.  -0.  -0.   0.  -0.   0.5]

Tests passed
